# Working with **Selenium webdriver** and **undetected-chromedriver** in Kaggle

<div class = 'alert alert-info' role = 'alert'>
  We can't directly use <strong>Selenium</strong> here. On direct use of <em>selenium</em> like <em>pandas</em>, it'll throw an error like the below.
</div>
<br>
<img src = 'https://user-images.githubusercontent.com/83589431/236695253-01af47e7-fb58-435c-bfbd-fb8f9473b992.png' width = 80%>

<strong>But</strong> there are ways to make it work. We'll see one version here. 🪄

# Installing libraries

- [Selenium](https://selenium-python.readthedocs.io/installation.html)
- [undetected-chromedriver](https://github.com/ultrafunkamsterdam/undetected-chromedriver)

In [1]:
!pip install selenium -q
!pip install undetected-chromedriver -q

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.11.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.
tensorflow-serving-api 2.11.0 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.3 which is incompatible.
pymc3 3.11.5 requires numpy<1.22.2,>=1.15.0, but you have numpy 1.23.5 which is incompatible.
pymc3 3.11.5 requires scipy<1.8.0,>=1.7.3, but you have scipy 1.9.3 which is incompatible.
librosa 0.10.0.post2 requires soundfile>=0.12.1, but you have soundfile 0.11.0 which is incompatible.
kfp 1.8.20 requires google-api-python-client<2,>=1.7.8, but you have google-api-python-client 2.86.0 which is incompatible.
kfp 1.8.20 requires PyYAML<6,>=5.3, but you have pyyaml 6.0 which is incompatible.
apache-beam 2.46.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.6 which is incompatible.


# Preparing Chrome to work in Kaggle

- We need to download chromedriver to kaggle.
- ChromeDriver can be downlodable from [chromedriver.chromium.org](https://chromedriver.chromium.org/).


<img src = 'https://user-images.githubusercontent.com/83589431/236694660-16ba1546-8f2d-4282-8593-68dac1cdd478.gif' width = 70%>

How to download *ChromeDriver* **Kaggle** ?
- Visit [chromedriver.chromium.org](https://chromedriver.chromium.org/).
- Check for the **Latest stable release** of ChromeDriver (at present v113.0.5672.63 is the latest)
- Copy the link of **`chromedriver_linux64.zip`** from **Index** page by right-clicking on it.

**Still doubut,** please chromedriver the above **GIF** for visual download instructions.

- Once we have the driver link, **`! wget <paste-the-copied-link-here>`**

In [2]:
# Downloading the chromedriver to Kaggle
!wget https://chromedriver.storage.googleapis.com/113.0.5672.63/chromedriver_linux64.zip
    
# Making that file executable
!chmod 755 /kaggle/working/chromedriver_linux64.zip

--2024-03-12 18:30:35--  https://chromedriver.storage.googleapis.com/113.0.5672.63/chromedriver_linux64.zip
Resolving chromedriver.storage.googleapis.com (chromedriver.storage.googleapis.com)... 108.177.11.207, 173.194.218.207, 142.251.162.207, ...
Connecting to chromedriver.storage.googleapis.com (chromedriver.storage.googleapis.com)|108.177.11.207|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7315061 (7.0M) [application/zip]
Saving to: ‘chromedriver_linux64.zip’

chromedriver_linux6 100%[===================>]   6.98M  --.-KB/s    in 0.06s   

2024-03-12 18:30:36 (127 MB/s) - ‘chromedriver_linux64.zip’ saved [7315061/7315061]



In [3]:
# Dowloading Google Chrome's latest stable version to Kaggle
!wget https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
    
# Installing Google Chrome
!sudo apt-get update
!sudo apt install ./google-chrome-stable_current_amd64.deb -y 

--2024-03-12 18:30:36--  https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
Resolving dl.google.com (dl.google.com)... 173.194.215.93, 173.194.215.190, 173.194.215.136, ...
Connecting to dl.google.com (dl.google.com)|173.194.215.93|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 106071384 (101M) [application/x-debian-package]
Saving to: ‘google-chrome-stable_current_amd64.deb’

google-chrome-stabl 100%[===================>] 101.16M   145MB/s    in 0.7s    

2024-03-12 18:30:37 (145 MB/s) - ‘google-chrome-stable_current_amd64.deb’ saved [106071384/106071384]

Hit:1 http://archive.ubuntu.com/ubuntu focal InRelease
Get:2 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Get:3 http://packages.cloud.google.com/apt gcsfuse-focal InRelease [1225 B]
Get:4 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Get:5 https://packages.cloud.google.com/apt cloud-sdk InRelease [6361 B]
Get:6 http://archive.ubuntu.com/ubun

**Let's verify the *Chrome* installation**

For this case the output should be ***Google Chrome 113.0.5672.63***

In [4]:
!google-chrome --version

Google Chrome 122.0.6261.111 


<img src = 'https://user-images.githubusercontent.com/83589431/237309858-e521dfca-667c-44d9-9914-7a5674c552d3.gif' width = 20%><br>

<div class = 'alert alert-success' role = 'alert'>
  Now it's the time to test the chrome drivers.
</div>

# Using Selenium-chromedriver

In [5]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service

from selenium.webdriver.common.by import By

In [6]:
options = Options()
options.add_argument('--no-sandbox')
options.add_argument('--disable-setuid-sandbox')
options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument('--disable-dev-shm-usage')
options.add_argument('--remote-debugging-port=9222')
service = Service('/kaggle/working/chromedriver/chromedriver')

**Let's  find out some text to confirm *Selenium* is working**

<img src = 'https://user-images.githubusercontent.com/83589431/237303838-89137892-0833-4370-900c-7fb65576fe2c.jpg' width =70%>

<img src = 'https://user-images.githubusercontent.com/83589431/237305592-05857628-624f-419c-883b-21ba4e681d98.png' width = 70%>


- In order to find the above marked text, we can see utilize the feature **XPATH**
- The text is present inside **`h1`** tag under **`class`** name **`badge-link__title`**.
- So the XPATH will be **`//h1[@class='badge-link__title']`**

<img src = 'https://user-images.githubusercontent.com/83589431/237309844-ffd07c38-ff26-4dcf-af95-3beda956ee84.gif' width = 20%><br>

<div class = 'alert alert-success' role = 'alert'>
  <p>We have successfully ran <strong>Selenium</strong> inside Kaggle.</p>
</div>

# Using undetected-chromedriver

In [7]:
import undetected_chromedriver as uc

In [8]:
driver = uc.Chrome(service = service, options = options)

#uc_driver.get('https://duckduckgo.com/')

In [9]:
!pip install gspread 

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.9/53.9 kB 2.4 MB/s eta 0:00:00


<img src = 'https://user-images.githubusercontent.com/83589431/237309844-ffd07c38-ff26-4dcf-af95-3beda956ee84.gif' width = 20%><br>

<div class = 'alert alert-success' role = 'alert'>
  <p>We have successfully ran <strong>undetected-chromedriver</strong> inside Kaggle.</p>
</div>

In [10]:
import pandas as pd
import gspread


In [11]:
gc = gspread.service_account(filename = "/kaggle/input/key-json/integral-hold-416907-82b173ff1425.json")


In [12]:
sheet = gc.open_by_url("https://docs.google.com/spreadsheets/d/108dnSSwlUtCGHmoAOyTI4FsuSnWzOZFF0dYuf8vodq4/edit?usp=sharing")
#sheet2 = gc.open_by_url("https://docs.google.com/spreadsheets/d/12M6NtFxw0-MtlwuHPMUckFR7DCdxubaNdoaGKj4Vld4/edit?usp=sharing")

In [13]:
sheet
worksheet = sheet.add_worksheet(title="Pennsylvania2", rows=1, cols=1)
#worksheet2 = sheet.add_worksheet(title="West Virginia2",rows=1,cols=1)

In [14]:
!pip install gspread pandas gspread-dataframe


In [15]:
from gspread_dataframe import set_with_dataframe

In [16]:
import sys
import time
import json
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [17]:
name = []
address = []
contact = []
link = []
best_match = []
zipcode = []
counter=0
for i in range(15001,19640):
    driver.get(f"https://www.mealsonwheelsamerica.org/signup/aboutmealsonwheels/find-programs?filter={i}")
    pagesource = driver.page_source
    soup = BeautifulSoup(pagesource, 'html.parser')
    boxs = soup.find_all('div', {'class': 'findmeal-result'})
    counter+=1
    if(counter%100==0):
        df = pd.DataFrame({"name":name,"address":address,"contact":contact,"link":link,"zipcode":zipcode})
        set_with_dataframe(worksheet,df)
    for box in boxs:
        zipcode.append(i)
        try:
            print("Name: ", box.find('h2').text.strip())
            name.append(box.find('h2').text.strip())
        except:
            name.append("NIL")
        try:
            print("Address: ",box.find('p').text.strip())
            address.append(box.find('p').text.strip())
        except:
            address.append('NIL')
        try:
            links = box.find_all('a')
            if(len(links)==2):
                print("Phone: ",links[1].text.strip())
                contact.append(links[1].text.strip())
            else:
                print("Phone: ",links[0].text.strip())
                contact.append(links[0].text.strip())
        except:
            contact.append("NIL")
        try:
            links = box.find_all('a')
            if(len(links)==1):
                link.append("NIL")
                continue
            print("Link: ",box.find('a')['href'])
            link.append(box.find('a')['href'])
        except:
            print("LINK NIL")
            link.append("NIL")
    

Name:  Five Points Meals on Wheels
Address:  1401 Gringo Rd, Aliquippa, PA  15001
Phone:  (724)378-8616
Name:  Allegheny County Department of Human Services Area Agency on Aging
Address:  2100 Wharton St Fl Second, Pittsburgh, PA  15203
Phone:  (412)350-4234
Link:  http://www.alleghenycounty.us/Human-Services/Programs-Services/Older-Adults.aspx
Name:  LifeSpan, Inc.
Address:  314 E 8Th Ave, Homestead, PA  15120
Phone:  (412)464-1300
Link:  http://www.lifespanpa.org
Name:  Lutheran Service Society of Western PA
Address:  440 Lincoln Ave, Pittsburgh, PA  15202
Phone:  (412)734-9330
Link:  http://www.lsswpa.org
Name:  Circle of Friends, Inc.
Address:  371 Linmore Ave, Baden, PA  15005
Phone:  (724)869-4224
Name:  Allegheny County Department of Human Services Area Agency on Aging
Address:  2100 Wharton St Fl Second, Pittsburgh, PA  15203
Phone:  (412)350-4234
Link:  http://www.alleghenycounty.us/Human-Services/Programs-Services/Older-Adults.aspx
Name:  Lutheran Service Society of Western P